In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as op

In [2]:
df1=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
df1.insert(1,"new",np.ones((42000,1)),True)

In [5]:
df1.head()

,label,new,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train=df1.iloc[:,1:].values
y_train=df1.iloc[:,[0]].values

In [7]:
df1.describe()

,label,new,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,1:] = sc.fit_transform(X_train[:,1:])

In [9]:
def sigmoid(b):
    return (1/(1+np.exp(-b)))

In [10]:
opt_theta=[]
for i in range(10):
    def costfunction(theta,A,b):
        (m,n)=A.shape
        theta=theta.reshape(n,1)
        b=b.reshape(m,1)
        term1=np.log(sigmoid(np.dot(A,theta)))
        term2=np.log(1-(sigmoid(np.dot(A,theta))))
        j1=(b*term1)
        j2=(1-b)
        j2=j2.reshape(m,1)
        j3=j2*term2
        j4=j1+j3
        J1=(-np.sum(j3)/m)
        J2=np.dot(theta.T,theta)
        J3=((4*J2)/(2*m))
        J=J1+J3
        return J
    def gradient(theta,A,b):
        (m,n)=A.shape
        theta=theta.reshape(n,1)
        b=b.reshape(m,1)
        term=sigmoid(np.dot(A,theta))-b
        j1=(np.dot(A.T,term)/m)
        j2=((4*theta)/m)
        j=j1+j2
        return j.flatten()
    y1=list(map(int,(y_train==i)))
    y1=np.array(y1)
    (m,n)=X_train.shape
    initial_theta=np.zeros((n,1))
    result=op.minimize(fun=costfunction,x0=initial_theta,args=(X_train,y1),method='CG',jac=gradient)
    opt_theta.append(result.x)


In [11]:
opt_theta=np.array(opt_theta)

In [13]:
def predict(A,b):
    (m,n)=A.shape
    h_of_theta=np.dot(A,b.T)
    y_pred=np.argmax(h_of_theta,axis=1)
    return y_pred

In [14]:
y_predictions=predict(X_train,opt_theta)

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [16]:
df2.insert(0,"new",np.ones((28000,1)),True)

In [17]:
X_test=df2.iloc[:,:].values

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test[:,1:] = sc.fit_transform(X_test[:,1:])

In [20]:
y_test_pred=predict(X_test,opt_theta)

In [21]:
y_test_pred=y_test_pred.reshape(28000,1)

In [22]:
prediction = pd.DataFrame(y_test_pred,index=range(1,28001),columns=['Label']).to_csv('prediction4.csv')